# Домашняя работа 7. Работа с категориальными признаками. ##
# Выполнил Дмитрий Головинов ##

### Оценивание и штрафы

Максимальная оценка — 10 баллов.

Не списывайте, иначе всем участникам обнулим :)

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка: 10**

In [40]:
print('Всем удачи!👒')

Всем удачи!👒


In [41]:
from __future__ import annotations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

## Работа с категориальными признаками (10 баллов)

Сейчас мы научимся обрабатывать категориальные переменные. Как вы уже знаете, закодировать их в виде столбика чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть, но модель попробует его выучить). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Подготовка данных.__

Загрузим данные [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Этот датасет содержит информацию о маркетинговой кампании какого-то банка, объектом в нем является телефонный звонок потенциальному клиенту с предложением некоторой услуги (утверждается, что это краткосрочный депозит), целевой переменной — ответ клиента (согласился ли он открыть депозит?). В качестве признакового описания используются характеристики клиента (образование, брак и т.д.), данные о звонке и различные экономические индикаторы — более подробная информация на страничке с датасетом.

In [29]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip bank-additional.zip
df = pd.read_csv('bank-additional/bank-additional-full.csv', sep=';')

--2023-12-01 19:16:23--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank-additional.zip.1’

bank-additional.zip     [  <=>               ] 434.15K  1.28MB/s    in 0.3s    

2023-12-01 19:16:24 (1.28 MB/s) - ‘bank-additional.zip.1’ saved [444572]

Archive:  bank-additional.zip
replace bank-additional/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [30]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


__Задание 1 (2 балла).__ Разделите выборку на обучающую и тестовую в соотношении 3:1. Зафиксируйте `random_state=777`, укажите значение параметра `stratify`. Один из столбцов (помимо таргета :) ) стоит сразу выкинуть из обучающей выборки. Какой? Не отказывайте себе.



Ответ: **Уберем столбец duration, поскольку он сильно влияет на таргет, но при этом неизвестен до звонка, а после звонка значение уже известно**

In [31]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df = df.drop(columns=['duration'])

train_df, test_df = train_test_split(df, test_size=0.25, random_state=777, stratify=df['y'])

train_df.head(), test_df.head()

(       age            job  marital    education  default housing loan  \
 23424   33  self-employed  married     basic.4y       no      no   no   
 39990   32    blue-collar  married     basic.4y       no     yes   no   
 9340    27        student   single  high.school  unknown      no   no   
 5365    34    blue-collar  married     basic.9y       no     yes   no   
 2797    41     technician  married     basic.9y       no      no   no   
 
          contact month day_of_week  campaign  pdays  previous     poutcome  \
 23424   cellular   aug         wed         7    999         0  nonexistent   
 39990  telephone   jun         tue         1    999         1      failure   
 9340   telephone   jun         fri         1    999         0  nonexistent   
 5365   telephone   may         fri         1    999         0  nonexistent   
 2797   telephone   may         wed         1    999         0  nonexistent   
 
        emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   

Закодируйте категориальные признаки с помощью `OrdinalEncoder`. Посчитайте качество (в этом задании будем работать c `AUC-PR`) при применении логистической регрессии. Здесь и далее для реализации последовательности этих действий (обработка признаков + обучение модели) используйте пайплайны. Замерьте время, потребовавшееся на обучение модели (с учетом кодирования признаков).

__Вопрос__: почему в данном задании мы выбрали метрикой именно `AUC-PR`, а не, к примеру, `AUC-ROC`?

__Ваш ответ__: **AUC-PR предпочтительнее в ситуациях, когда классы в данных распределены неравномерно (т.е., когда имеет место значительный дисбаланс классов). AUC-ROC (Area Under the Receiver Operating Characteristic Curve) может давать завышенную оценку производительности модели в случае сильного дисбаланса классов, так как она более сосредоточена на истинно положительных и ложноположительных результатах, не учитывая количество фактических положительных случаев. В контексте маркетинговых данных, где часто классы могут быть неравномерно распределены (например, небольшое количество клиентов согласится на предложение), AUC-PR является более информативной метрикой.**

In [32]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time

# Создаем массив категориальных признаков (без таргета)
categorical_features = [col for col in train_df.columns if train_df[col].dtype == 'object' and col != 'y']

categorical_transformer = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), categorical_features)
    ], remainder='passthrough')

model_pipeline = Pipeline(steps=[
    ('ordinal_encoder', categorical_transformer),
    ('logistic_regression', LogisticRegression())
])

label_encoder = LabelEncoder()

X_train = train_df.drop('y', axis=1)
X_test = test_df.drop('y', axis=1)
y_train = label_encoder.fit_transform(train_df['y'])
y_test = label_encoder.transform(test_df['y'])

# Тренируем модель и засекаем время
start_time= time.time()
model_pipeline.fit(X_train, y_train)
training_time = time.time() - start_time

# Предсказание на тестовой выборке
y_pred_prob = model_pipeline.predict_proba(X_test)[:, 1]

# AUC-PR
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
auc_pr = auc(recall, precision)

training_time, auc_pr

(0.4309117794036865, 0.41672358050945824)

__Задание 2 (2 балла).__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели. Измерьте время, потребовавшееся на кодирование категориальных признаков и обучение модели.



In [33]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from sklearn.preprocessing import OneHotEncoder

one_hot_transformer = ColumnTransformer(
    transformers=[
        ('one_hot', OneHotEncoder(), categorical_features)
    ], remainder='passthrough')

# Создаем пайплайн с OneHotEncoder и LogisticRegression
one_hot_model_pipeline = Pipeline(steps=[
    ('one_hot_encoder', one_hot_transformer),
    ('logistic_regression', LogisticRegression())
])

# Тренируем модель и засекаем время
start_time_one_hot = time.time()
one_hot_model_pipeline.fit(X_train, y_train)
training_time_one_hot = time.time() - start_time_one_hot

y_pred_prob_one_hot = one_hot_model_pipeline.predict_proba(X_test)[:, 1]

# AUC-PR
precision_one_hot, recall_one_hot, _ = precision_recall_curve(y_test, y_pred_prob_one_hot)
auc_pr_one_hot = auc(recall_one_hot, precision_one_hot)

training_time_one_hot, auc_pr_one_hot
# заняло больше чем original encoding, но качество выросло

(0.7610275745391846, 0.4364337725071667)

Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирования категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 3 (2 балла).__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущими экспериментами (с учетом кодирования признаков). Заметили ли вы что-то интересное?

In [34]:
label_encoder = LabelEncoder()
train_df['y_encoded'] = label_encoder.fit_transform(train_df['y'])
test_df['y_encoded'] = label_encoder.transform(test_df['y'])

# Создаем копии датафреймов
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

# Счётчики для каждой категории на основе обучающего набора данных
for feature in categorical_features:
    means = train_df_copy.groupby(feature)['y_encoded'].mean()
    train_df_copy[feature + '_mean_target'] = train_df_copy[feature].map(means)
    test_df_copy[feature + '_mean_target'] = test_df_copy[feature].map(means)

# Удаляем оригинальные категориальные признаки
train_df_copy.drop(columns=categorical_features, inplace=True)
test_df_copy.drop(columns=categorical_features, inplace=True)

# Подготовка данных для обучения модели
X_train_mean_target = train_df_copy.drop(['y', 'y_encoded'], axis=1)
y_train_mean_target = train_df_copy['y_encoded']
X_test_mean_target = test_df_copy.drop(['y', 'y_encoded'], axis=1)
y_test_mean_target = test_df_copy['y_encoded']

# Пайплайн для логистической регрессии
mean_target_model_pipeline = Pipeline([
    ('logistic_regression', LogisticRegression())
])

start_time_mean_target = time.time()
mean_target_model_pipeline.fit(X_train_mean_target, y_train_mean_target)
training_time_mean_target = time.time() - start_time_mean_target

# Предсказание и расчет AUC-PR
y_pred_prob_mean_target = mean_target_model_pipeline.predict_proba(X_test_mean_target)[:, 1]
precision_mean_target, recall_mean_target, _ = precision_recall_curve(y_test_mean_target, y_pred_prob_mean_target)
auc_pr_mean_target = auc(recall_mean_target, precision_mean_target)

(training_time_mean_target, auc_pr_mean_target)


(0.34548377990722656, 0.41745408839324294)

**Интересно, наверное, что счётчики предоставили сравнительно высокое качество модели при меньшем времени обучения по сравнению с one-hot-кодированием, что делает этот метод эффективным с точки зрения вычислительных ресурсов, особенно на больших датасетах с множеством категориальных признаков.**

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем «утечку» целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении счетчика для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
1. Вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени).
2. Вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации).
3. Вносить шум в посчитанные признаки.

__Задание 4 (2 балла).__ Реализуйте корректное вычисление счётчиков самым простым способом — добавлением шума к значениям (постарайтесь найти баланс между борьбой с переобучением и сохранением полезности признаков). Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.
Какие плюсы и минусы использования счётчиков по сравнению с one-hot-кодированием можно отметить?

In [35]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from sklearn.model_selection import KFold
import numpy as np

# Функция для добавления шума к среднему значению целевой переменной
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

# Копии датафреймов для новых признаков с шумом
train_df_noisy = train_df.copy()
test_df_noisy = test_df.copy()

# Вычисляем средние значения с учетом шума для каждой категории
noise_level = 0.01  # Уровень шума можно подбирать в зависимости от задачи
for feature in categorical_features:
    # Вычисляем среднее значение целевой переменной по категориям
    overall_mean = train_df_noisy[feature].map(train_df_noisy.groupby(feature)['y_encoded'].mean())
    # Добавляем шум к средним значениям
    train_df_noisy[feature + '_mean_target'] = add_noise(overall_mean, noise_level)
    # Маппим средние значения с шумом на тестовый набор, используя средние значения из обучающего набора
    test_df_noisy[feature + '_mean_target'] = test_df_noisy[feature].map(train_df_noisy.groupby(feature)['y_encoded'].mean())

# Удаляем оригинальные категориальные признаки
train_df_noisy.drop(columns=categorical_features, inplace=True)
test_df_noisy.drop(columns=categorical_features, inplace=True)

# Обучение логистической регрессии на преобразованных данных
X_train_noisy = train_df_noisy.drop(['y', 'y_encoded'], axis=1)
y_train_noisy = train_df_noisy['y_encoded']
X_test_noisy = test_df_noisy.drop(['y', 'y_encoded'], axis=1)
y_test_noisy = test_df_noisy['y_encoded']

# Пайплайн для логистической регрессии
noisy_model_pipeline = Pipeline([
    ('logistic_regression', LogisticRegression())
])

# Замер времени обучения модели
start_time_noisy = time.time()
noisy_model_pipeline.fit(X_train_noisy, y_train_noisy)
training_time_noisy = time.time() - start_time_noisy

# Предсказание на тестовом наборе
y_pred_prob_noisy = noisy_model_pipeline.predict_proba(X_test_noisy)[:, 1]

# Расчет AUC-PR
precision_noisy, recall_noisy, _ = precision_recall_curve(y_test_noisy, y_pred_prob_noisy)
auc_pr_noisy = auc(recall_noisy, precision_noisy)

(training_time_noisy, auc_pr_noisy)



(0.3655359745025635, 0.4173502262805444)

**Вывод: качество такое же, времени требуется меньше**

__Задание 5 (2 балла).__ В данных имеется признак «возраст клиента». Сейчас мы интерпретируем его как числовой, что в общем случае для линейной модели может быть неверной гипотезой. Тем не менее, у этого признака есть довольно много уникальных значений (сколько?), поэтому применять к нему one-hot кодирование может оказаться излишним. Попробуйте закодировать возраст с помощью счетчиков. Стало ли лучше?



**Всего 76 уникальных значений (вот код)**

In [36]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
unique_ages = train_df['age'].nunique()
unique_ages


76

In [37]:
train_df['y_encoded'] = label_encoder.fit_transform(train_df['y'])
test_df['y_encoded'] = label_encoder.transform(test_df['y'])

# Теперь создадим новые копии датафреймов для кодирования
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

# Счётчики с добавлением шума
for feature in categorical_features:
    # Добавление шума
    mean_target = train_df_copy.groupby(feature)['y_encoded'].transform('mean')
    train_df_copy[feature + '_mean_target'] = mean_target + np.random.normal(0, 0.01, size=mean_target.shape)
    # Применение средних значений из обучающего набора к тестовому набору
    test_df_copy[feature + '_mean_target'] = test_df_copy[feature].map(train_df_copy.groupby(feature)['y_encoded'].mean())

# Удаление исходных категориальных признаков
train_df_copy.drop(columns=categorical_features, inplace=True)
test_df_copy.drop(columns=categorical_features, inplace=True)

# Определение признаков и целевой переменной для обучения модели
X_train_mean_encoded = train_df_copy.drop(['y', 'y_encoded'], axis=1)
y_train_mean_encoded = train_df_copy['y_encoded']
X_test_mean_encoded = test_df_copy.drop(['y', 'y_encoded'], axis=1)
y_test_mean_encoded = test_df_copy['y_encoded']

# Создание и обучение пайплайна
mean_encoded_pipeline = Pipeline([
    ('logistic_regression', LogisticRegression())
])

# Замер времени обучения модели
start_time_mean_encoded = time.time()
mean_encoded_pipeline.fit(X_train_mean_encoded, y_train_mean_encoded)
training_time_mean_encoded = time.time() - start_time_mean_encoded

# AUC-PR
y_pred_mean_encoded = mean_encoded_pipeline.predict_proba(X_test_mean_encoded)[:, 1]
precision_mean_encoded, recall_mean_encoded, _ = precision_recall_curve(y_test_mean_encoded, y_pred_mean_encoded)
auc_pr_mean_encoded = auc(recall_mean_encoded, precision_mean_encoded)

(training_time_mean_encoded, auc_pr_mean_encoded)


(0.3240053653717041, 0.4173403256401724)

**Стало быстрее, но качество упало**

Можно пойти и в обратную сторону. У нас есть признаки «месяц и день недели» для звонка. Попробуйте интерпретировать их как числовые (месяц от 0 до 12, дни недели от 0 до 4). Стало ли лучше в этот раз?

***(Наверное имелось ввиду от 0 до 6 про дни недели)***

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

In [38]:
X_train_numerical = train_df.drop(['y', 'y_encoded'], axis=1)
y_train_numerical = train_df['y_encoded']
X_test_numerical = test_df.drop(['y', 'y_encoded'], axis=1)
y_test_numerical = test_df['y_encoded']

categorical_features_to_remove = [col for col in X_train_numerical.columns if X_train_numerical[col].dtype == 'object']

X_train_numerical_filtered = X_train_numerical.drop(categorical_features_to_remove, axis=1)
X_test_numerical_filtered = X_test_numerical.drop(categorical_features_to_remove, axis=1)

# Создание и обучение пайплайна с логистической регрессией
numerical_pipeline_filtered = Pipeline([
    ('logistic_regression', LogisticRegression(max_iter=1000))
])

# Замер времени обучения модели
start_time_numerical_filtered = time.time()
numerical_pipeline_filtered.fit(X_train_numerical_filtered, y_train_numerical)
training_time_numerical_filtered = time.time() - start_time_numerical_filtered

# Предсказание и расчет AUC-PR на тестовом наборе
y_pred_numerical_filtered = numerical_pipeline_filtered.predict_proba(X_test_numerical_filtered)[:, 1]
precision_numerical_filtered, recall_numerical_filtered, _ = precision_recall_curve(y_test_numerical, y_pred_numerical_filtered)
auc_pr_numerical_filtered = auc(recall_numerical_filtered, precision_numerical_filtered)

(training_time_numerical_filtered, auc_pr_numerical_filtered)

(0.41239047050476074, 0.42457516028811654)

**Качество и время улучшилось - победа**